In [1]:
!python -m venv venv

In [4]:
!venv/Scripts/pip install ipykernel

'venv' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
!venv\Scripts\pip install pandas pymc3 matplotlib

INFO: pip is looking at multiple versions of pymc3 to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
     ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
     ----------- ---------------------------- 0.5/1.8 MB 2.1 MB/s eta 0:00:01
     ----------------------- ---------------- 1.0/1.8 MB 2.2 MB/s eta 0:00:01
     ---------------------------------- ----- 1.6/1.8 MB 2.3 MB/s eta 0:00:01
     ---------------------------------------- 1.8/1.8 MB 1.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [33 lines of output]
  C:\Users\etale\AppData\Local\Temp\pip-install-cgez4lwk\theano-pymc_9e83f19179ea4c398ee23d9f5ae0fc03\versioneer.py:564: SyntaxWarning: invalid escape sequence '\s'
    mo = re.search(r'=\s*"(.*)"', line)
  Traceback (most recent call last):
    File "C:\Users\etale\Documents\week10\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
      main()
      ~~~~^^
    File "C:\Users\etale\Documents\week10\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
      json_out["return_val"] = hook(**hook_input["kwargs"])
                               ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\etale\Documents\week10\venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
      return hook(

In [2]:
!venv\Scripts\pip freeze > requirements.txt

In [3]:
!git init

'git' is not recognized as an internal or external command,
operable program or batch file.
